# Import Packages

In [ ]:
import sys
import os
import numpy as np
import scanpy as sc
import pandas as pd
import anndata as ad
import episcanpy.api as epi
from matplotlib import pyplot as plt

from wp2.gff_analyser import gffClasses,gffBuilder

# modules from sctoolbox
from sctoolbox import annotation
import sctoolbox.atac as atac
import sctoolbox.calc_overlap_pct as overlap
from sctoolbox import celltype_annotation

## Set environmental variables

In [ ]:
GTF_PATH = 'homo_sapiens.104.mainChr.gtf'  # gtf file to use for peaks annotation
#INPUT_PATH = '/home/rstudio/data/anndata'  # path where the h5ad object is saved
WORKING_DIR = '/mnt/workspace_stud/catlas_ref/'
INPUT = WORKING_DIR + 'cellXcCRE/'
OUTPUT_PATH = '/home/rstudio/testdata'  # path where processed adata file can be save
FRAG_FILE = '/home/stud2/test/frag_sorted.bed'
#FRAG_FILE = '/mnt/workspace_stud/catlas_ref/frag/colon_sigmoid_SM-AZPYO_rep1_fragments.bed.gz'  # path to fragments file
BAM_FILE = '/mnt/workspace_stud/jdetlef/ENC-1JKYN-191-SM-JF1O8_snATAC_colon_sigmoid_Rep1.bam'  # path to bam file
H5AD = 'stomach_SM-JF1O3.h5ad'
#H5AD = 'ENC-1JKYN-146-SM-A8CPH_snATAC_esophagus_muscularis_mucosa_Rep1.h5ad'  # name of the h5ad file

## Import H5AD file

In [ ]:
adata = epi.read_h5ad(INPUT + H5AD)
adata

## Generate GFF files

In [ ]:
object_list = gffBuilder.build_gff3_class(file=GTF_PATH)

for element in object_list:
    features = element.count_features()
    element.generate_feature_gtf(Gffdata_list=object_list, feature_keys=features)

### Prepare Var_names

In [ ]:
names = []
for name in adata.var_names:
   names.append(name)

for i, n in enumerate(names):
   tmp = names[i].replace(':', '_')
   tmp = tmp.replace('-', '_')
   tmp = tmp.split('_')
   names[i] = '_'.join([tmp[0], tmp[-2], tmp[-1]])

adata.var_names = pd.Index(names)

### Prepare obs_names

In [4]:
names = []
for name in adata.obs_names:
   names.append(name)

for i, n in enumerate(names): # colon_sigmoid_SM-JF1O8_1+
    tmp = names[i].split('+')
    names[i] = tmp[-1]

adata.obs_names = pd.Index(names, name='index')

NameError: name 'adata' is not defined

### Calculate Overlap

In [ ]:
overlap.pct_fragments_in_promoters(adata, input_dir='out/', fragments_file=FRAG_FILE, bam_file=None, cb_col=None, species=None)

## Initial Violin Plots

In [6]:
epi.pl.violin(adata, ['n_total_fragments'])
epi.pl.violin(adata, ['n_fragments_in_promoters'])
epi.pl.violin(adata, ['pct_fragments_in_promoters'])

NameError: name 'epi' is not defined

In [8]:
## Lowpass filter